# Label-Free in vivo Quantification of the Vascular Network in Murine Colitis using Transrectal Absorber Guide Raster-Scanning Optoacoustic Mesoscopy (TAG-RSOM)

The following code provides a pipeline to pick up results generaded by the Fiji-pipeline, calculate the parameters and to build the tabels to be analysed in Prism (statistic software)

In [21]:
import os
import pandas as pd
import numpy as np

# Choose Working Directory

In [22]:
folder_paths = [r"D:\newPipeline_v4.0\final_5_medianF_after_Seg\4_AT",
                r"D:\newPipeline_v4.0\final_5_medianF_after_Seg\5_RF",
                r"D:\newPipeline_v4.0\final_5_medianF_after_Seg\6_VF"]
path_analysis = "D:/newPipeline_v4.0/final_5_medianF_after_Seg/Auswertung/"

# Normalized Number of Connected Components
The number of networks with two or more connected branches (vessels) in an image is defined as the number of components that have one or more junctions. This information is calculated by Fiji's analyse skeleton function, called in the macro "skeletonize_analyse_and_save_Results.ijm" and saved as file ending with "_results.csv".

    ~~Connected Components (CC) is natural number has no unit~~

To normalize the CC it is devided by the area of the used colon 2D image in cm². The colon area is saved as number of pixels in a file ending with "\_vesselArea.csv" when calling any segmentation macro.

(AREA_COLON_pixel * 20µm * 20µm) / 100000000 = AREA_COLON_cm²

    Normalized Number of Connected Components (CC_n) has the unit (cm^-2)

In [23]:
def calculate_NNCC(f):
    #df_CC =  pd.DataFrame(columns=['Name', 'Value'])
    df_NNCC =  pd.DataFrame(columns=['Name', 'Value'])
    
    
    for skeleton_name in sorted([file for file in os.listdir(f) if file.endswith("_results.csv")]): # Iterate throu individuals
        # Open nessesary files
        df_skeleton = pd.read_csv(os.path.join(f, skeleton_name))
        df_area = pd.read_csv(os.path.join(f, skeleton_name.replace("_results.csv", "_vesselArea.csv")))
        AREA_COLON_pixel = df_area.iloc[0]["Area"] # Returns number of pixels
        AREA_COLON_CM2 = AREA_COLON_pixel * 0.000004
        
        number_CC = 0
        for index, row in df_skeleton.iterrows(): # Iterate throu components (networks)
            if(row['# Junctions'] != 0): # Def. Connected Components
                number_CC += 1
        
        #df_CC.loc[len(df_CC)] = [skeleton_name.replace("_results.csv", ""), number_CC]
        df_NNCC.loc[len(df_NNCC)] = [skeleton_name.replace("_results.csv", ""), number_CC / AREA_COLON_CM2]
    
    
    #df_CC.to_csv(f+"/Results"+"/Connected Components.csv", header=False, index=False)
    df_NNCC.to_csv(f+"/Results"+"/Normalized Number of Connected Components.csv", header=False, index=False)

# Normalized Number of Branches
The number of branches (vessels) in a skeleton is calculated by Fiji's analyse skeleton function, called in the macro "skeletonize_analyse_and_save_Results.ijm" and saved as file ending with "_BranchInfo.csv".

To normalize the NB it is devided by the area of the used colon 2D image in cm². The colon area is saved as number of pixels in a file ending with "\_vesselArea.csv" when calling any segmentation macro.

(AREA_COLON_pixel * 20µm * 20µm) / 100000000 = AREA_COLON_cm²

    Normalized Number of Branches (NNB) has the unit (cm^-2)

In [24]:
def calculate_NNB(f):
    df_NNB =  pd.DataFrame(columns=['Name', 'Value'])
    
    
    for skeleton_name in sorted([file for file in os.listdir(f) if file.endswith("_results.csv")]): # Iterate throu individuals
        # Open nessesary files
        df_area = pd.read_csv(os.path.join(f, skeleton_name.replace("_results.csv", "_vesselArea.csv")))
        AREA_COLON_pixel = df_area.iloc[0]["Area"] # Returns number of pixels
        AREA_COLON_CM2 = AREA_COLON_pixel * 0.000004
        df_branches = pd.read_csv(os.path.join(f, skeleton_name.replace("_results.csv", "_BranchInfo.csv")))
        
        df_NNB.loc[len(df_NNB)] = [skeleton_name.replace("_results.csv", ""), df_branches.shape[0] / AREA_COLON_CM2]
        
        
    df_NNB.to_csv(f+"/Results"+"/Normalized Number of Branches.csv", header=False, index=False)

# Number of Branches in Largest Component
The number of branches (vessels) in a skeleton is calculated by Fiji's analyse skeleton function, called in the macro "skeletonize_analyse_and_save_Results.ijm" and saved as file ending with "_BranchInfo.csv".

    The Number of Branches in Largest Component (LC_NB) has no unit

In [25]:
def calculate_LC_NB(f):
    LC_NB =  pd.DataFrame(columns=['Name', 'Value'])
    
    
    for skeleton_name in sorted([file for file in os.listdir(f) if file.endswith("_results.csv")]): # Iterate throu individuals
        # Open nessesary files
        df_branches = pd.read_csv(os.path.join(f, skeleton_name.replace("_results.csv", "_BranchInfo.csv")))
        df_skeleton = pd.read_csv(os.path.join(f, skeleton_name))
        
        number_CC = 0
        index_LC = 0
        for index, row in df_skeleton.iterrows(): # Iterate throu components (networks)
            if(row['# Junctions'] != 0): # Def. Connected Components
                number_CC += 1
                index_LC = index
        
        df_branches_MC = df_branches[df_branches['Skeleton ID'] == index_LC+1]
        LC_NB.loc[len(LC_NB)] = [skeleton_name.replace("_results.csv", ""), df_branches_MC.shape[0]]
        
        
    LC_NB.to_csv(f+"/Results"+"/Largest Component_Number of Branches.csv", header=False, index=False)

# Length of Largest Component
The largest component is calculated as the number of pixels representing junction-, end-, or slab-pixels. This information is calculated by Fiji's analyse skeleton function, called in the macro "skeletonize_analyse_and_save_Results.ijm" and saved as file ending with "_results.csv".

    Largest Component (CC) is a length and has the unit (mm)
    
Calculated by number of pixels in LC * 20 µm / 1000

In [26]:
def calculate_LC_L(f):
    df_LC_L =  pd.DataFrame(columns=['Name', 'Value'])
    
    
    for skeleton_name in sorted([file for file in os.listdir(f) if file.endswith("_results.csv")]): # Iterate throu individuals
        # Open nessesary files
        df_skeleton = pd.read_csv(os.path.join(f, skeleton_name))
        
        tmp_LC = 0
        for index, row in df_skeleton.iterrows(): # Iterate throu components (networks)
            if(row['# End-point voxels'] + row['# Junction voxels'] + row['# Slab voxels'] > tmp_LC):
                tmp_LC = row['# End-point voxels'] + row['# Junction voxels'] + row['# Slab voxels']
                
        df_LC_L.loc[len(df_LC_L)] = [skeleton_name.replace("_results.csv", ""), tmp_LC * 0.02]
    
    
    df_LC_L.to_csv(f+"/Results"+"/Largest Components_Total Length.csv", header=False, index=False)

# Normalized Network Length
The Normalized Network Length is calculated by deviding the total number of junction-, end-, or slab-pixels by the colon area. This information is calculated by Fiji's analyse skeleton function, called in the macro "skeletonize_analyse_and_save_Results.ijm" and saved as file ending with "_results.csv". The colon area is saved as number of pixels in a file ending with "\_vesselArea.csv" when calling any segmentation macro.

    Normalized Network Length (NNL) has the unit (µm)

In [27]:
def calculate_NNL(f):
    df_NNL =  pd.DataFrame(columns=['Name', 'Value'])
    
    
    for skeleton_name in sorted([file for file in os.listdir(f) if file.endswith("_results.csv")]): # Iterate throu individuals
        # Open nessesary files
        df_skeleton = pd.read_csv(os.path.join(f, skeleton_name))
        df_area = pd.read_csv(os.path.join(f, skeleton_name.replace("_results.csv", "_vesselArea.csv")))
        AREA_COLON_pixel = df_area.iloc[0]["Area"] # Returns number of pixels
        
        number_pixels = 0
        for index, row in df_skeleton.iterrows(): # Iterate throu components (networks)
            number_pixels += row['# End-point voxels'] + row['# Junction voxels'] + row['# Slab voxels']
        
        df_NNL.loc[len(df_NNL)] = [skeleton_name.replace("_results.csv", ""), number_pixels/(AREA_COLON_pixel*20)]
    
    
    df_NNL.to_csv(f+"/Results"+"/Normalized Network Length.csv", header=False, index=False)

# Normalized Vessel Area
the normalized vessel area is calculated by the percentage of non-zero pixels in the colon area. This information is saved as % in a file ending with "\_vesselArea.csv" when calling any segmentation macro.

    Normalized Vessel Area (NVA) has the unit (%)

In [28]:
def calculate_NVA(f):
    df_NVA =  pd.DataFrame(columns=['Name', 'Value'])
    
    
    for skeleton_name in sorted([file for file in os.listdir(f) if file.endswith("_results.csv")]): # Iterate throu individuals
        # Open nessesary files
        df_area = pd.read_csv(os.path.join(f, skeleton_name.replace("_results.csv", "_vesselArea.csv")))
        
        df_NVA.loc[len(df_NVA)] = [skeleton_name.replace("_results.csv", ""), df_area.iloc[0]["%Area"]]
    
    
    df_NVA.to_csv(f+"/Results"+"/Normalized Vessel Area.csv", header=False, index=False)

# Average Vessel Diameter
The average vessel diameter is calculated by deviding the total vessel area by the total vessel length.

    Average Vessel Diameter (AVD) has the unit (µm)

In [29]:
def calculate_AVD(f):
    df_AVD =  pd.DataFrame(columns=['Name', 'Value'])
    
    
    for skeleton_name in sorted([file for file in os.listdir(f) if file.endswith("_results.csv")]): # Iterate throu individuals
        # Open nessesary files
        df_skeleton = pd.read_csv(os.path.join(f, skeleton_name))
        df_area = pd.read_csv(os.path.join(f, skeleton_name.replace("_results.csv", "_vesselArea.csv")))
        AREA_VESSEL_pixels = (df_area.iloc[0]["Area"] / 100) * df_area.iloc[0]["%Area"] # Returns number of pixels
        
        number_pixels = 0
        for index, row in df_skeleton.iterrows(): # Iterate throu components (networks)
            number_pixels += row['# End-point voxels'] + row['# Junction voxels'] + row['# Slab voxels']
        
        df_AVD.loc[len(df_AVD)] = [skeleton_name.replace("_results.csv", ""), (AREA_VESSEL_pixels*20)/number_pixels]
    
    
    df_AVD.to_csv(f+"/Results"+"/Average Vessel Diameter.csv", header=False, index=False)

# Average Vessel Diameter (Distance Map)
The average vessel diameter is calculated based on the histogram of the geographical distance map.

    Average Vessel Diameter (Distance Map) (AVD_DM) has the unit (µm)

In [30]:
def calculate_AVD_DM(f):
    df_AVD_DM =  pd.DataFrame(columns=['Name', 'Value'])
    
    
    for skeleton_name in sorted([file for file in os.listdir(f) if file.endswith("_results.csv")]): # Iterate throu individuals
        # Open nessesary files
        AVD_DM = pd.read_csv(os.path.join(f, skeleton_name.replace("_results.csv", "_vesselThickness.csv")))
        # Drop first and zero lines
        AVD_DM = AVD_DM.drop(0)
        AVD_DM = AVD_DM[AVD_DM['Count'] != 0]
        AVD_DM_mm = (AVD_DM['Distance'] * AVD_DM['Count']*2*20).sum() / AVD_DM['Count'].sum()
        
        df_AVD_DM.loc[len(df_AVD_DM)] = [skeleton_name.replace("_results.csv", ""), AVD_DM_mm]
    
    
    df_AVD_DM.to_csv(f+"/Results"+"/Average Vessel Diameter_Distance Map.csv", header=False, index=False)

# Normalized Blood Volume (Distance Map)
The normalized Blood Volume is calculated based on the geographic distance map relative to the colon area imaged.

    Normalized Blood Volume (Distance Map) (NBV_DM) has the unit (µm)

In [31]:
def calculate_NBV_DM(f):
    df_NBV_DM =  pd.DataFrame(columns=['Name', 'Value'])
    
    
    for skeleton_name in sorted([file for file in os.listdir(f) if file.endswith("_results.csv")]): # Iterate throu individuals
        # Open nessesary files
        df_area = pd.read_csv(os.path.join(f, skeleton_name.replace("_results.csv", "_vesselArea.csv")))
        AREA_COLON_pixel = df_area.iloc[0]["Area"] # Returns number of pixels
        AVD_DM = pd.read_csv(os.path.join(f, skeleton_name.replace("_results.csv", "_vesselThickness.csv")))
        # Drop first and zero lines
        AVD_DM = AVD_DM.drop(0)
        AVD_DM = AVD_DM[AVD_DM['Count'] != 0]
        AVD_DM_mm = ((AVD_DM['Distance']*20)**2 * 3.1416 *AVD_DM['Count']*20).sum() / (AREA_COLON_pixel*20*20)
        
        df_NBV_DM.loc[len(df_NBV_DM)] = [skeleton_name.replace("_results.csv", ""), AVD_DM_mm]
    
    
    df_NBV_DM.to_csv(f+"/Results"+"/Normalized Blood Volume_Distance Map.csv", header=False, index=False)

# Perimeter-Area Ratio
The perimeter-area ratio is calculated as the mean value of the ratios of length and area anclosed by the component (conhex hull)

    Perimeter Area Ratio (PAR) has the unit (µm)

In [32]:
def calculate_PAR(f):
    df_PAR =  pd.DataFrame(columns=['Name', 'Value'])
    
    
    for skeleton_name in sorted([file for file in os.listdir(f) if file.endswith("_results.csv")]): # Iterate throu individuals
        df_skeleton = pd.read_csv(os.path.join(f, skeleton_name))
        df_convex_hulls = pd.read_csv(os.path.join(f, skeleton_name.replace("_results.csv", "_convex_hulls.csv")))
        df = pd.concat([df_skeleton, df_convex_hulls], axis=1)
        df_peri_area_ratio = (((df["# End-point voxels"] + df["# Junction voxels"] + df["# Slab voxels"])*20) / df["Area"]).mean()
        df_PAR.loc[len(df_PAR)] = [skeleton_name.replace("_results.csv", ""), df_peri_area_ratio]
    
    
    df_PAR.to_csv(f+"/Results"+"/Perimeter Area Ratio.csv", header=False, index=False)

# Fractal Dimension
The fractal dimension is calculated by the box-counting method (using Fiji)

    Fractal Dimension (FD) has no unit

In [33]:
def calculate_FD(f):
    df_FD =  pd.DataFrame(columns=['Name', 'Value'])
    
    
    for skeleton_name in sorted([file for file in os.listdir(f) if file.endswith("_results.csv")]): # Iterate throu individuals
        df_fd = pd.read_csv(os.path.join(f, skeleton_name.replace("_results.csv", "_box_count.csv")))
        df_FD.loc[len(df_FD)] = [skeleton_name.replace("_results.csv", ""), df_fd["D"].mean()] # Calling .mean() is a dirty workaround to avoid getting dtaype and row number
    
    
    df_FD.to_csv(f+"/Results"+"/Fractal Dimension.csv", header=False, index=False)

# Cord-to-length ratio
The cord-to-length ratio (CLR) is the ratio between the Eukledian distance of the ends and the length of a vessel. A straight vessel has a CLR of 1. Circles have to be pruned to prevent infinite values.

    Cord to Length Ratio (CLR) has not unit

In [34]:
def calculate_CLR(f):
    df_CLR =  pd.DataFrame(columns=['Name', 'Value'])
    
    
    for skeleton_name in sorted([file for file in os.listdir(f) if file.endswith("_results.csv")]): # Iterate throu individuals
        df_branches = pd.read_csv(os.path.join(f, skeleton_name.replace("_results.csv", "_BranchInfo.csv")))
        clr = []
        for index, row in df_branches.iterrows():
            if row['Branch length'] > 1 and row['Euclidean distance'] > 1:
                clr.append(row['Branch length'] / row['Euclidean distance'])
        df_CLR.loc[len(df_CLR)] = [skeleton_name.replace("_results.csv", ""), sum(clr)/len(clr)]
    
    
    df_CLR.to_csv(f+"/Results"+"/Cord to Length Ratio.csv", header=False, index=False)

# Cord-to-length ratio in largest component
See cord-to-length ratio

    Cord to Length Ratio in Largest Component (CLR) has not unit

In [35]:
def calculate_LC_CLR(f):
    df_LC_CLR =  pd.DataFrame(columns=['Name', 'Value'])
    
    
    for skeleton_name in sorted([file for file in os.listdir(f) if file.endswith("_results.csv")]): # Iterate throu individuals
        df_skeleton = pd.read_csv(os.path.join(f, skeleton_name))
        df_branches = pd.read_csv(os.path.join(f, skeleton_name.replace("_results.csv", "_BranchInfo.csv")))
        
        LC = 0
        LC_index = 0
        for index, row in df_skeleton.iterrows():
            if(row['# End-point voxels'] + row['# Junction voxels'] + row['# Slab voxels'] > LC):
                LC = row['# End-point voxels'] + row['# Junction voxels'] + row['# Slab voxels']
                LC_index = index
        
        clr = []
        for index, row in df_branches[df_branches['Skeleton ID'] == LC_index+1].iterrows():
            if row['Branch length'] > 1 and row['Euclidean distance'] > 1:
                clr.append(row['Branch length'] / row['Euclidean distance'])
        
        df_LC_CLR.loc[len(df_LC_CLR)] = [skeleton_name.replace("_results.csv", ""), sum(clr)/len(clr)]
    
    
    df_LC_CLR.to_csv(f+"/Results"+"/Largest Component_Cord to Length Ratio.csv", header=False, index=False)

# Calculate the results

In [37]:
for f in folder_paths:
    output_path = os.path.join(f, "Results")
    if not os.path.exists(output_path):
        try: os.makedirs(output_path)
        except OSError as e: print(f"Error creating folder '{output_path}': {e}")
    
    calculate_NNCC(f) # Normalized Number of Connected Components
    calculate_NNB(f) # Normalized Number of Branches
    calculate_LC_NB(f) # Number of Branches in Largest Component
    calculate_LC_L(f) # Length of Largest Components
    calculate_NNL(f) # Normalized Network Length
    calculate_NVA(f) # Normalized Vessel Area
    calculate_AVD(f) # Average Vessel Diameter
    calculate_AVD_DM(f) # Average Vessel Diameter_Distance Map
    calculate_NBV_DM(f) # Normalized Blood Volume_Distance Map
    calculate_PAR(f) # Perimeter Area Ratio
    calculate_FD(f) # Fractal Dimension
    calculate_CLR(f) # Cord to Length Ratio
    calculate_LC_CLR(f) # Cord to Length Ratio in Largest Component

# Merge data tables into one master table

In [46]:
all_df = pd.DataFrame()

for p in folder_paths:
    path = p + "/Results"
    files = os.listdir(path)
    # create a list of DataFrames from .csv files
    dfs = []
    for i in files:
        if i.endswith(".csv"): # exclude non-.csv
            #print(path + "/" + i)
            tmp = pd.read_csv(path + "/" + i, header=None)
            tmp.columns = ['name', i.replace(".csv", "")] # rename colums in 'name' and 'paramerer'-> filename
            dfs.append(tmp)
            
    df = pd.concat(dfs, axis=1) # merge all frames into one
    df = df.loc[:, ~df.columns.duplicated()] # drop duplicates
    
    #Split the column 'name' into single variables
    ########################################################
    
    # Split the first column using "_" as delimiter
    split_df = df['name'].str.split('_', expand=True)
    # Rename the columns in the split DataFrame
    split_df.columns = ['study', 'ID', 'Day', 'contrast']
    
    # concat new columns and drop "name"
    concat_df = pd.concat([split_df, df.drop(columns=['name'])], axis=1)
    
    if p.endswith("VF"):
        concat_df.insert(4, 'Method', "VF")
    elif p.endswith("RF"):
        concat_df.insert(4, 'Method', "RF")
    elif p.endswith("AT"):
        concat_df.insert(4, 'Method', "AT")
    else:
        print("An error occurred! Worng Folder name?")
        
    all_df = pd.concat([all_df, concat_df], ignore_index=True)
    
    
    
all_df.to_excel(path_analysis + "all.xlsx", index=False)
#all_df.to_csv(path_analysis + "all.csv", index=False)

# Export tables for statistic program

In [47]:
df = pd.read_excel(path_analysis + "all.xlsx")
m = "VF"

collectives = [
            [
                "Method",
                [
                    ['AT', 'hc2', 0, "AT", "w"],
                    ['VF', 'hc2', 0, "VF", "w"],
                    ['RF', 'hc2', 0, "RF", "w"]
                ]
            ],
            [
                "1xDSS",
                [
                    ['HC Day 0', 'hc2', 0, m, "w"],
                    ['HC Day 7', 'hc2', 9, m, "w"],
                    ['1xDSS Day 0', 'a3d', 0, m, "w"],
                    ['1xDSS Day 9', 'a3d', 9, m, "w"]
                ]
            ],
            [
                "2xDSS",
                [
                    ['2xDSS Day 0', 'l2d', 0, m, "w"],
                    ['2xDSS Day 7', 'l2d', 7, m, "w"],
                    ['2xDSS Day 21', 'l2d', 21, m, "w"],
                    ['2xDSS Day 28', 'l2d', 28, m, "w"]
                ]
            ]
         ]

In [48]:
df_sorted = df.sort_values(by=["study", 'Day', 'ID'])

for parameter in df.columns[5:]:
    for c in collectives:
        columns = []
        
        for s in c[1]:
            df_Method = df_sorted[df_sorted['Method'] == s[3]] 
            df_Method_Contrast = df_Method[df_Method['contrast'] == s[4]]
            df_M_C = df_Method_Contrast.drop('Method', axis=1)
            df_M_C = df_M_C.drop('contrast', axis=1)
            parameters_to_keep = ["study", "ID", "Day", parameter]
            tmp_df = df_M_C[parameters_to_keep]
            
            column = tmp_df[(tmp_df['study'] == s[1]) & (tmp_df['Day'] == s[2])][parameter] # for validation incert comment -> #[parameter]
            column.reset_index(drop=True, inplace=True)
            column.name = s[0] + " (n = " + str(len(column)) + ")"
            columns.append(column)
            
        pd.concat(columns, axis=1).to_excel(path_analysis + "/" + c[0] + "_" + parameter + ".xlsx", index=False)
        pd.concat(columns, axis=1).to_csv(path_analysis + "/" + c[0] + "_" + parameter + ".csv", index=False)

# Test Microvessels

In [12]:
folder_paths = [r"D:\newPipeline_v4.0\test_microvasculatur\4_AT",
                r"D:\newPipeline_v4.0\test_microvasculatur\5_1_RF",
                r"D:\newPipeline_v4.0\test_microvasculatur\7_test_all",
                r"D:\newPipeline_v4.0\test_microvasculatur\6_VF"]
path_analysis = "D:/newPipeline_v4.0/test_microvasculatur/Auswertung/"


def calculated_MV(f):
    df_MV =  pd.DataFrame(columns=['Name', 'Value'])
    
    
    for name in sorted([file for file in os.listdir(f) if file.endswith("_avg_intensity_betweenvessels.csv")]): # Iterate throu individuals
        df_histo = pd.read_csv(os.path.join(f, name))
        MV = df_histo.drop(0)
        mean_MV = (MV['Value']*MV['Count']).sum() / MV['Value'].sum()
        
        df_MV.loc[len(df_MV)] = [name.replace("_avg_intensity_betweenvessels.csv", ""), mean_MV]
    
    
    df_MV.to_csv(f+"/Results"+"/Micro Vessels.csv", header=False, index=False)

In [13]:
for f in folder_paths:
    output_path = os.path.join(f, "Results")
    if not os.path.exists(output_path):
        try: os.makedirs(output_path)
        except OSError as e: print(f"Error creating folder '{output_path}': {e}")
    calculated_MV(f)

# Tag depth

In [54]:
folder_paths = [r"D:\newPipeline_v4.0\test_depth\rLabs_export"]
path_analysis = "D:/newPipeline_v4.0/test_depth/Auswertung/"


def calculated_Depth(f):
    df_depth =  pd.DataFrame(columns=['Name', 'Value'])
    
    
    for name in sorted([file for file in os.listdir(f) if file.endswith(".csv")]): # Iterate throu individuals
        data_file = pd.read_csv(os.path.join(f, name))
        # ,Area,X,Y,Slice
        mean_depth = ((4*data_file['Y'].sum()) / (data_file['Y'].shape[0])) / 1000 # *4 -> µm    # / 100 -> cm
        df_depth.loc[len(df_depth)] = [name.replace("_segmentSize_50.csv", ""), mean_depth]
    
    
    df_depth.to_csv(f+"/Results"+"/Depth_in_cm.csv", header=False, index=False)

In [55]:
for f in folder_paths:
    output_path = os.path.join(f, "Results")
    if not os.path.exists(output_path):
        try: os.makedirs(output_path)
        except OSError as e: print(f"Error creating folder '{output_path}': {e}")
    calculated_Depth(f)

In [ ]:
all_df = pd.DataFrame()

for p in folder_paths:
    path = p + "/Results"
    files = os.listdir(path)
    # create a list of DataFrames from .csv files
    dfs = []
    for i in files:
        if i.endswith(".csv"): # exclude non-.csv
            #print(path + "/" + i)
            tmp = pd.read_csv(path + "/" + i, header=None)
            tmp.columns = ['name', i.replace(".csv", "")] # rename colums in 'name' and 'paramerer'-> filename
            dfs.append(tmp)
            
    df = pd.concat(dfs, axis=1) # merge all frames into one
    df = df.loc[:, ~df.columns.duplicated()] # drop duplicates
    
    #Split the column 'name' into single variables
    ########################################################
    
    # Split the first column using "_" as delimiter
    split_df = df['name'].str.split('_', expand=True)
    # Rename the columns in the split DataFrame
    split_df.columns = ['study', 'ID', 'Day', 'contrast']
    
    # concat new columns and drop "name"
    concat_df = pd.concat([split_df, df.drop(columns=['name'])], axis=1)
        
    all_df = pd.concat([all_df, concat_df], ignore_index=True)
    
all_df.to_excel(path_analysis + "depth.xlsx", index=False)

# Tag angle

In [84]:
import math
folder_paths = [r"D:\newPipeline_v4.0\test_depth\rLabs_export"]
path_analysis = "D:/newPipeline_v4.0/test_depth/Auswertung/"


def calculated_Angle(f):
    df_angle =  pd.DataFrame(columns=['Name', 'Value'])
    
    
    for name in sorted([file for file in os.listdir(f) if file.endswith(".csv")]): # Iterate throu individuals
        data_file = pd.read_csv(os.path.join(f, name))
        sign = 1
        b = 50*len(data_file) / 2
        a = data_file['Y'][1] - data_file['Y'][len(data_file)-1] # second value alway TAG coordinate
        if a < 0:
            sign = -sign
            a = -a
        
        
        df_angle.loc[len(df_angle)] = [name.replace("_segmentSize_50.csv", ""), sign * math.degrees(math.tan((4*a)/(20*b)))]
    
    
    df_angle.to_csv(f+"/Results"+"/Angle_in_Degree.csv", header=False, index=False)

In [85]:
for f in folder_paths:
    output_path = os.path.join(f, "Results")
    if not os.path.exists(output_path):
        try: os.makedirs(output_path)
        except OSError as e: print(f"Error creating folder '{output_path}': {e}")
    calculated_Angle(f)

614
593
492
482
426
409
411
457
704
700
488
508
521
510
570
638
564
518
542
555
502
458
614
629
460
428
549
560
752
734
569
544
523
495
533
556
505
478
570
582
622
619
579
594
470
447
463
495
562
517
439
451
564
503
412
442
615
590
520
534
489
489
574
575
511
502
514
536
808
808
510
505
614
600
504
519
678
672
642
643
440
423
620
590
439
421
496
498
565
534
525
538
442
436
633
597
589
550
459
469
451
420
453
478
628
542
490
520
539
520
521
535
435
466
545
570
570
560
508
507
563
598
499
521
398
422
409
385
501
522
477
496
580
562
455
447
538
524


In [86]:
all_df = pd.DataFrame()

for p in folder_paths:
    path = p + "/Results"
    files = os.listdir(path)
    # create a list of DataFrames from .csv files
    dfs = []
    for i in files:
        if i.endswith(".csv"): # exclude non-.csv
            #print(path + "/" + i)
            tmp = pd.read_csv(path + "/" + i, header=None)
            tmp.columns = ['name', i.replace(".csv", "")] # rename colums in 'name' and 'paramerer'-> filename
            dfs.append(tmp)
            
    df = pd.concat(dfs, axis=1) # merge all frames into one
    df = df.loc[:, ~df.columns.duplicated()] # drop duplicates
    
    #Split the column 'name' into single variables
    ########################################################
    
    # Split the first column using "_" as delimiter
    split_df = df['name'].str.split('_', expand=True)
    # Rename the columns in the split DataFrame
    split_df.columns = ['study', 'ID', 'Day', 'contrast']
    
    # concat new columns and drop "name"
    concat_df = pd.concat([split_df, df.drop(columns=['name'])], axis=1)
        
    all_df = pd.concat([all_df, concat_df], ignore_index=True)
    
all_df.to_excel(path_analysis + "angle.xlsx", index=False)